<a href="https://colab.research.google.com/github/swiggy123/flask/blob/main/dlbs_intro_cnn_grad_cam_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-On Deep Learning Workbook: Training a CNN for Classification and Insights into a Blackbox Model

Lecturer: Susanne Suter (susanne.suter@fhnw.ch)

This notebook was inspired by https://appliedmldays.org/workshops/machine-learning-for-smart-dummies

# Before You Start
1) Click on "Copy to Drive". A Google account is needed for that.

2) Select "Runtime" -> "Change runtime type" -> "Hardware accelerator" : "GPU"

Now you work on your personal copy of the notebook. Let's get started!

# Gender Recognition using a CNN

In this assignment, we will create and tune a convolutional neural network (CNN) that is able to detect the gender of a given face image. That is, we model a classification task. For simplicity, we define our outputs as male or female.

You will be gently guided through every step of the CNN predictor creation. Assignment tasks are explicitly marked with <font color='blue'>Task</font>.

The ground truth data contains of the labels (answers) inserted into the CNN that are used for training. That is, the ground truth represents the true y values.

The selected ground truth data is not perfect, hence, this will allow you to grasp important aspects about the ground truth data generation.

During the assignment you can moreover reflect about what effects biases in the selection of ground truth data have on the model prediction.

## Importing Data

Retrieve the data set used for this exercise. This time, we will import the data for this exercise directly from a git repository. 

The ownership of the data is: "Labeled faces in the wild"
http://vis-www.cs.umass.edu/lfw/
Gary B. Huang, Manu Ramesh, Tamara Berg, and Erik Learned-Miller. 
Labeled Faces in the Wild: A Database for Studying Face Recognition in
Unconstrained Environments. University of Massachusetts, Amherst, 
Technical Report 07-49, October, 2007.

The face images are for teaching purposes prepared in a repository splitting male and female faces such that they are already sorted in two directories for our two classes to predict.


In [ ]:
# Import data for male and female labelled faces (includes independent benchmark data)

!rm -fr faces* # removes the data directory in case it already exists
!git clone https://github.com/susuter/faces_red.git faces

In [ ]:
!ls -ltr faces

The loaded data contains three folders with faces images in jpg format. You can explore them by browsing the directories next to this notebook file.

* Folder `female` contains images from female personalities
* Folder `male` contains images from male personalities
* Folder `benchmark` contains independent test images

In [ ]:
import os
base_dir = "faces"
m_dir = os.path.join(base_dir, "male")
f_dir = os.path.join(base_dir, "female")

print(m_dir, f_dir)

In [ ]:
# Show an image of each gender to make sure that the data is correctly loaded 
# and to get an idea of how the images look like
# Feel free to load other images yourself

import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io

img_f = io.imread("faces/female/AJ_Cook_0001.jpg")
img_m = io.imread("faces/male/AJ_Lamas_0001.jpg")

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_f)
ax1.set_title("Example image females")
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_m)
ax2.set_title("Example image males")


In [ ]:
# Shape of the images
# (pixel rows, pixel columns, colors)

print( "Shape of female image: ")
print( img_f.shape )

print( "Shape of male image: ")
print( img_m.shape )

In [ ]:
# Print the number of source images available for each dataset

import glob

def print_file_count_in_dir(dir_name, msg="", extension=".jpg"):
    print(msg, len(glob.glob(os.path.join(dir_name, "*" + extension))))

print_file_count_in_dir(m_dir, "# male cases: ")
print_file_count_in_dir(f_dir, "# female cases: ")

In [ ]:
# method to randomly select n images from the male and female data sources

import os, random, shutil

# Set random seed for reproducibility
seed = 7
random.seed(seed)

def randomly_select_n_images(in_path, out_path, n):
  in_files = os.listdir(in_path)
  path, dirs, out_files = next(os.walk(out_path))
  while len(out_files) < n:    
    choice = random.choice(in_files)
    src = os.path.join(in_path, choice)
    dst = os.path.join(out_path, choice)
    if os.path.isfile(src): 
      shutil.copy(src, dst)
    path, dirs, out_files = next(os.walk(out_path))

In [ ]:
!rm -r faces/female/selected
!rm -r faces/male/selected
!mkdir faces/female/selected
!mkdir faces/male/selected

# you may ignore the message 
# "rm: cannot remove ...: No such file or directoy"
# it happens when you execute that cell for the first time

### Data Selection
<font color='blue'>Task</font>: Select an appropriate number of images for both classes. 

In [ ]:
### BEGIN SOLUTION
# choose your own number of randomly selected images
n_f = 800
n_m = 800

### END SOLUTION

f_dir_sel = os.path.join(f_dir, 'selected')
m_dir_sel = os.path.join(m_dir, 'selected')

randomly_select_n_images(f_dir, f_dir_sel, n_f)
randomly_select_n_images(m_dir, m_dir_sel, n_m)

# output the number of selected images 
print_file_count_in_dir(f_dir_sel)
print_file_count_in_dir(m_dir_sel)

## Data Preparation
In this part, we want to make sure that the data is correctly formatted.
For example, we need each image to be of the same size with respect to the images dimensions (WIDTH and HEIGHT) and the number of color channels (DEPTH). At the same time, we add the prepared image to a list with our desired file format.

In [ ]:
# Generate the image lists for both classes, female and male
import glob

image_list_f = glob.glob(os.path.join(f_dir_sel, '*.jpg'))
image_list_m = glob.glob(os.path.join(m_dir_sel,'*.jpg'))
print(len(image_list_f), len(image_list_m))

In [ ]:
# Define the function that pre-processes the images for the training

import numpy as np
from skimage.transform import rescale, resize, downscale_local_mean

# This function does the resize and adds the image to the training set
def img_preprocessing(image_list, label, X_, y_):

    i = 0
    for image in image_list:
    
        if i%100 == 0: print("pre-processing image ",i," ...")
    
        img = io.imread(image)

        img = np.array(img)

        # Resize the image; make sure all images have same size
        pr = 250 # pixel rows (HEIGHT)
        pc = 250 # pixel columns (WIDTH)
        img = resize(img, (pr, pc), anti_aliasing=True)

        if i == 0:
            WIDTH, HEIGHT, DEPTH = np.array(img).shape
            print("image size:",WIDTH,HEIGHT)

        # Show the first 5 images
        if i < 5:
            plt.imshow(img)
            plt.show()
    
        # Append the pre-processed images to the training set
        X_.append(img)
        y_.append(label)
    
        i = i+1

<font color='blue'>Task</font>: Process your selected data so that it can be used later for model training. Incl. conversion to Numpy arrays. You may use existing functions.



In [ ]:
# Initialize the training set
X_v0 = [] # images
y_v0 = [] # labels

# Pre-process images and add them to training set

### BEGIN SOLUTION
img_preprocessing(image_list_f, [1,0], X_v0, y_v0)
img_preprocessing(image_list_m, [0,1], X_v0, y_v0)
### END SOLUTION

# Change format of the training set from list to numpy array
X = np.array(X_v0) # images
y = np.array(y_v0) # labels

print("Training set is ready.")

### Data and Labels Check
<font color='blue'>Task</font>: Control your data selection by displaying 2 images of each class as an image and specifying the associated label. You may use existing functions.

In [ ]:
# Define a function that shows an image from the image_set
def show_img(image_set, i):
    img1 = image_set[i, :, :]
    plt.imshow(img1)
    plt.show()

In [ ]:
# Show example images from the training set and check their labels
# Label -> what we are predicting
# Label [1 0] -> female
# Label [0 1] -> male

def show_label(i):
  show_img(X,i)
  print("Label: ",  y[i,:])

# Task: select image indices
# Hint 1: the index range depends on your chosen number of selected augmented images
# Hint 2: the female images have the lower indices since they were added to the list first

### BEGIN SOLUTION
show_label(10)
show_label(510)
### END SOLUTION

In [ ]:
# Print X (images) and y (labels) shapes
print(X.shape,y.shape)

In [ ]:
# Optionally print X (images) and y (labels) values
print("Images X: ",X)
print("Labels y: ",y)

## Split Data into Training Set and Validation Set

During the training, the CNN needs certain validation data to compute the loss. What in return makes it possible to update the parameters (weights) of the trained CNN in order to make it perform better. 

Therefore, we split our ground truth data into a training set and a validation set. Typically, we choose about 20% of the ground truth as validation set. 

The terminology of the validation set can vary. It is also called test set. 

### Training Set Split
<font color='blue'>Task</font>: Perform the training data set split so that 15% of the data is in the validation set (test set). You may use existing functions.


In [ ]:
import numpy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Split the data from the preprocessed set into a train and test set
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.20)

print('X_train shape: ',X_train.shape)
print('X_validation shape: ',X_validation.shape)
print('y_train shape: ',y_train.shape)
print('y_validation shape: ',y_validation.shape)

print('Train and validation datasets are ready!')

## CNN Model Definition
In this section, we define the architecture of our CNN model. That means, we define how many and what type of layers the CNN has. We furthermore define other parameters such as the activation function or our regularization (dropout). 

<font color='blue'>Task</font>: Configure your own CNN model by adjusting the following parameters: 
* Select suitable activation functions for each layer of the CNN.
* Select an appropriate Loss, which is suitable for a classification problem.
* Select Stochastic Gradient Descent as the optimizer for backpropagation.
You may use existing functions.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy

# Set random seed for reproducibility
seed = 7
numpy.random.seed(seed)

def define_model(num_classes,epochs):
    # Create the CNN model
    model = Sequential()

    ### BEGIN SOLUTION
    # Task: enter activation functions

    # Layer 1 (convolutional plus max pooling)
    model.add(Conv2D(4, (5, 5), input_shape=(X.shape[1], X.shape[2], 3), padding='same', activation='relu', kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2 (convolutional plus max pooling)
    model.add(Conv2D(4, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 3 (convolutional plus max pooling)
    model.add(Conv2D(4, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Task: Optionally add additional convolutional layers
    # ...

    # Additional dense layers (fully connected)    
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
 
    # Task: choose (custome-defined) optimizer
    #lrate = 0.005
    #decay = lrate/epochs
    #sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    #adam = Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay, amsgrad=False)
    adam = Adam()
    
    # Prepares the model and defines the loss and optimizer
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    ### END SOLUTION
    
    print(model.summary())
    
    return model


<font color='blue'>Task</font>: Select a suitable number of epochs. 


In [ ]:
# Define the duration of the training process, which is given in epochs
# In each epoch the model learns once the whole dataset

### BEGINN SOLUTION

# Task: choose a number of epochs that the model should be trained for
epochs = 20

### END SOLUTION

#Create the model
model=define_model(2,epochs)

## CNN Training

In [ ]:
# Train the model
history=model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=epochs, batch_size=32)

# Plot accuracy vs epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Plot cost function vs epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model cost function')
plt.ylabel('cost function')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Final evaluation of the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("______________________________")
print("Validation set accuracy: %.2f%%" % (scores[1]*100))
print("______________________________")
# Save the model to a json file
model_json = model.to_json()
with open("model_faces_v1.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_faces_v1.h5")

<font color='blue'>Task</font>: Perform your training three times and compare the results. What values do you get? How stable is the training? How satisfied are you with your choice of number of epochs? Do the results in the graphs meet your expectations? 




# Performance Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix

import seaborn as sns
plt.style.use('seaborn-muted')

# Predictions on the validation sample
y_pred_validation = (model.predict(X_validation)>0.5).astype('int32')

# Predictions on the training sample
y_pred_train = (model.predict(X_train)>0.5).astype('int32')

print('Confusion matrix for the train set:')
cm_train = confusion_matrix(y_train[:,0], y_pred_train[:,0])
sns.heatmap(cm_train, annot=True, fmt='d')
plt.xlabel('predicted label')
plt.ylabel('true label')

print('[[true negatives, false negatives]')
print('[false positives, true positives]]')

<font color='blue'>Task</font>: What do the true negatives, false positives, false negatives and true positives mean with respect to our binary classification problem of predicting females and males on photos?


In [ ]:
print('Confusion matrix for the validation set:')
cm_validation = confusion_matrix(y_validation[:,0], y_pred_validation[:,0])
sns.heatmap(cm_validation, annot=True, fmt='d')
plt.xlabel('predicted label')
plt.ylabel('true label')

## Independent Benkmark Test Set


Now, the trained CNN model is evaluated on an independent test data set, which was not used for to calculated the loss during the training.

To differentiate from the terminology test set, which was used during the training, we use here the terminology benchmark test set.

In [ ]:
# Prepare the benchmark test set

import glob

# Generate the image lists for both benchmark classes, female and male
# The source data was downloaded previously from the repository
image_list_bench_f = glob.glob('faces/benchmark/female/*.jpg')
image_list_bench_m = glob.glob('faces/benchmark/male/*.jpg')

# Initialize the benchmark set
X_bench_v0 = [] # images
y_bench_v0 = [] # labels

# Pre-process images and add them to benchmark set
print("pre-processing female images and add them to benchmark set...")
img_preprocessing(image_list_bench_f, [1,0], X_bench_v0, y_bench_v0)
print("pre-processing male images and add them to benchmark set...")
img_preprocessing(image_list_bench_m, [0,1], X_bench_v0, y_bench_v0)

# Change format of the benchmark set from list to numpy array
X_bench = np.array(X_bench_v0) # images
y_bench = np.array(y_bench_v0) # labels

print("Benchmark set is ready.")


In [ ]:
# Predict the images of the benchmark test set
p_bench = (model.predict(X_bench)) # probabilities
p_bench_c = (p_bench>0.5).astype('int32') # classification

## Performance of Benchmark Set

<font color='blue'>Task</font>: Implement two functions for the manual calculation of precision and recall. 


In [ ]:
# note: handling division by zero for readability does not need to be implemented

# precision (positive predictive value)
### BEGINN SOLUTION
def calc_precision(tp,fp):
  return (tp)/(tp+fp)
### END SOLUTION

# recall or sensitivity (true positive rate)
### BEGINN SOLUTION
def calc_recall(tp,fn):
  return (tp)/(tp+fn)
### END SOLUTION



<font color='blue'>Task</font>: Calculate the confusion matrix for the benchmark data set and print the true positives etc. Then use your implemented recall and precision functions and verify your results using sklearn.



In [ ]:
# Calculate performance

from sklearn.metrics import precision_score, recall_score, confusion_matrix

### BEGIN SOLUTION
y_true = y_bench[:,0]
y_pred = p_bench_c[:,0]

print('Confusion matrix for the benchmark test set:')
cf = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('[[true negatives (tn), false positives (fp)]')
print('[false negatives (fn), true positives (tn)]]')
print('[[true males (tn), false females (fp)]')
print('[false males (fn), true females (tn)]]')
print(cf)
print('tn, fp, fn, tp')
print(tn, fp, fn, tp)

n = y_bench.shape[0]
tn = cf[0,0]
tp = cf[1,1]
fp = cf[0,1]
fn = cf[1,0]
print(tn, fp, fn, tp)


### END SOLUTION

print("Precision (manual): ",calc_precision(tp,fp))
# Only report results for the class specified by pos_label. 
# This is applicable only if targets (y_{true,pred}) are binary.
ps = precision_score(y_true, y_pred, average='binary')
print('Precision score sklearn (binary): ', ps)
# If None, the scores for each class are returned
ps = precision_score(y_true, y_pred, average=None)
print('Precision score sklearn (none): ', ps)

print("Recall (manual): ",calc_recall(tp,fn))
rs = recall_score(y_true, y_pred, average='binary')
print('Recall score sklearn (binary): ', rs)
rs = recall_score(y_true, y_pred, average=None)
print('Recall score sklearn (none): ', rs)


In [ ]:
print('Confusion matrix for the validation set:')
cm_validation = confusion_matrix(y_bench[:,0], p_bench_c[:,0])
sns.heatmap(cm_validation, annot=True, fmt='d')
plt.xlabel('predicted label')
plt.ylabel('true label')

<font color='blue'>Task</font>: For 2 images of each class, output the probability of belonging to a class as calculated by the CNN. Select 1 example each that shows clear results and 1 example each that shows uncertain predictions. 

In [ ]:
def print_prediction(X_, y_,p_,i):
  print("Label: ",y_[i,:])
  print("Prediction (class): ", np.round(p_[i,:]))
  print("Prediction (prob): ",p_[i,:])

def show_example_prediction(X_, y_,p_,i):
  show_img(X_,i)
  print_prediction(X_, y_,p_,i)

### BEGIN SOLUTION

# Hint: the probabilites were already calculated above

# females (smaller indices - depends on your total ground truth size)
show_example_prediction(X_bench,y_bench,p_bench,0)
show_example_prediction(X_bench,y_bench,p_bench,1)

# males (larger indices - depends on your total ground truth size)
show_example_prediction(X_bench,y_bench,p_bench,110) 
show_example_prediction(X_bench,y_bench,p_bench,150) 

### END SOLUTION

# Check With Your Own Photos

In [ ]:
# Run this cell to upload your foto in jpg or jpeg format
from google.colab import files
uploaded = files.upload()

In [ ]:
# List of the uploaded jpeg images
image_list_test = glob.glob('*.jp*g')
print(image_list_test)

In [ ]:
# Initialize the test set
X_test2 = []
y_test2 = []

# Pre-process images and append to the test set
img_preprocessing(image_list_test, [], X_test2, y_test2)

In [ ]:
# Prepare the dataset
X_test22 = np.array(X_test2)
print(X_test22.shape)

X_test23 = X_test22.reshape(X_test22.shape[0], X_test22.shape[1], X_test22.shape[2], 3)

# Get the predictions
p_test23 = (model.predict(X_test23)>0.5).astype('int32')
print(p_test23)

In [ ]:
# Loop over the uploaded images and show predictions
for i in range(0,len(image_list_test)):
  show_img(X_test23,i)
  print("Prediction:  ",p_test23[i,:])
  print("_______________________")

# Interpretable ML Analysis: Grad-CAM
Adapted from: https://towardsdatascience.com/understand-your-algorithm-with-grad-cam-d3b62fce353

<a id="grad_cam"></a>

## Gradient-weighted Class Activation Mapping (Grad-CAM)

Grad-CAM ist eine Methode, die Gradienten aus der letzten Schicht eines convolutional Layers eines CNNs extrahiert, um die Regionen auf den Inputbildern visuell hervorzuheben, welche den grössten Einfluss auf die vorhergesagte Wahrscheintlichkeit einer Klasse haben.

Links:
* __[Grad-CAM Tutorial](https://towardsdatascience.com/understand-your-algorithm-with-grad-cam-d3b62fce353)__ (inkl. Link zu Colab Notebook)
* __[Github Repository für Grad-CAM mit PyTorch](https://github.com/jacobgil/pytorch-grad-cam)__ (inkl. diversen Erweiterungen zu Grad-CAM)

In [ ]:
from tensorflow.keras.models import Model
import cv2

def GradCam(model, img_array, layer_name, eps=1e-8):
    '''
    Creates a grad-cam heatmap given a model and a layer name contained with that model
    
    Args:
      model: tf model
      img_array: (img_width x img_width) numpy array
      layer_name: str

    Returns 
      uint8 numpy array with shape (img_height, img_width)

    '''

    gradModel = Model(
			inputs=[model.inputs],
			outputs=[model.get_layer(layer_name).output,
				model.output])
    
    with tf.GradientTape() as tape:
			# cast the image tensor to a float-32 data type, pass the
			# image through the gradient model, and grab the loss
			# associated with the specific class index
            inputs = tf.cast(img_array, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, 0]
		# use automatic differentiation to compute the gradients
    grads = tape.gradient(loss, convOutputs)
    
    # compute the guided gradients
    castConvOutputs = tf.cast(convOutputs > 0, "float32")
    castGrads = tf.cast(grads > 0, "float32")
    guidedGrads = castConvOutputs * castGrads * grads
		# the convolution and guided gradients have a batch dimension
		# (which we don't need) so let's grab the volume itself and
		# discard the batch
    convOutputs = convOutputs[0]
    guidedGrads = guidedGrads[0]
    # compute the average of the gradient values, and using them
		# as weights, compute the ponderation of the filters with
		# respect to the weights
    weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
  
    # grab the spatial dimensions of the input image and resize
		# the output class activation map to match the input image
		# dimensions
    (w, h) = (img_array.shape[2], img_array.shape[1])
    heatmap = cv2.resize(cam.numpy(), (w, h))
		# normalize the heatmap such that all values lie in the range
		# [0, 1], scale the resulting values to the range [0, 255],
		# and then convert to an unsigned 8-bit integer
    numer = heatmap - np.min(heatmap)
    denom = (heatmap.max() - heatmap.min()) + eps
    heatmap = numer / denom
    # heatmap = (heatmap * 255).astype("uint8")
		# return the resulting heatmap to the calling function
    return heatmap


def sigmoid(x, a, b, c):
    return c / (1 + np.exp(-a * (x-b)))

def superimpose(img_bgr, cam, thresh, emphasize=False):
    
    '''
    Superimposes a grad-cam heatmap onto an image for model interpretation and visualization.
    
    Args:
      image: (img_width x img_height x 3) numpy array
      grad-cam heatmap: (img_width x img_width) numpy array
      threshold: float
      emphasize: boolean

    Returns 
      uint8 numpy array with shape (img_height, img_width, 3)

    '''
    heatmap = cv2.resize(cam, (img_bgr.shape[0], img_bgr.shape[1]))
    if emphasize:
        heatmap = sigmoid(heatmap, 50, thresh, 1)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    hif = .8
    superimposed_img = heatmap * hif + img_bgr
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img_rgb = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
    return superimposed_img_rgb

<font color='blue'>Tasks</font>: 
* Zeige die Grad-CAM Resultate für verschiedene Beispielbilder an (i= )
  * Wie verhalten sich die Predictions für unterschiedliche Bilderklassen wie Frauen, Männern, Sportler, Nicht-Sportler etc.
* Analysiere und diskutiere die Grad-CAM Resultate der unterschiedlichen CNN-Layers
  * Schaue dir die Gradienten Maps der vierschiedenen Blöcke an und interpretiere die Features
  * Welche Muster beobachtest du? Welche Layers sind relevant für die Klassifikation?
* Kannst du mittels Grad-CAM die Architektur des CNNs verbessern?
* Verwende optional andere XAI Methoden wie SHAP zur Analyse

In [ ]:
print(X_bench.shape)
# Task: select image for analysis
i = 3
img = X_bench[i, :, :] # Task: play with images
print(img.shape)
print_prediction(X_bench,y_bench,p_bench,i)

## Grad-CAM heatmap for the last convolutional layer in the model

# Task: add layer name you want to interpret grad-CAM for
# Task: find relevant CNN layers for the classification of the images
layer_name1 = 'conv2d' 

grad_cam = GradCam(model,np.expand_dims(img, axis=0),layer_name1)
# Task: optimize parameters for visualization
grad_cam_superimposed = superimpose(img, grad_cam, 0.35, emphasize=True)

layer_name2 = 'conv2d_1' 
grad_cam = GradCam(model,np.expand_dims(img, axis=0),layer_name2)
# Task: optimize parameters for visualization
grad_cam_superimposed2 = superimpose(img, grad_cam, 0.35, emphasize=True)

layer_name3 = 'conv2d_2' 
grad_cam = GradCam(model,np.expand_dims(img, axis=0),layer_name3)
# Task: optimize parameters for visualization
grad_cam_superimposed3 = superimpose(img, grad_cam, 0.35, emphasize=True)


plt.figure(figsize=(15, 5))
ax = plt.subplot(1, 4, 1)
plt.imshow(img)
plt.title('Original image')
ax = plt.subplot(1, 4, 2)
plt.imshow(grad_cam_superimposed)
plt.title(layer_name1 + ' grad-CAM')
ax = plt.subplot(1, 4, 3)
plt.imshow(grad_cam_superimposed2)
plt.title(layer_name2 + ' grad-CAM')
ax = plt.subplot(1, 4, 4)
plt.imshow(grad_cam_superimposed3)
plt.title(layer_name3 + ' grad-CAM')
plt.axis('off')
plt.tight_layout()

Congratulations for completing this exercise!


#Deluxe: Data Augmentation
Data augmentation is performed in order to make your ground truth dataset more diverse such that the computed algorithm is more robust to new data - not used for the training. Typical data augmentation modifications include adjusting brightness, color, contrast, distorions, adding noise, zooming, rotations, or mirroring.

There are various ready made data augmentor tools available. In case you need to augment also your labels, make sure you choose an augmentor tool that is able to do so. The one presented here can only augment raw images.  

<font color='blue'>Optional task</font>: integrate data augmentation into your deep learning model.*kursiver Text*

In [ ]:
!pip install Augmentor

In [ ]:
# Import Augmentor (library to create more distorted images)
import Augmentor

# Define a function to create augmented images images
# Task: change parameters of augmentor
def build_augmented_images(path_folder, n_samples):
    p = Augmentor.Pipeline(path_folder)
    p.rotate(probability=0.3, max_left_rotation=4, max_right_rotation=4)
    p.zoom(probability=0.3, min_factor=0.7, max_factor=1.2) 
    p.random_distortion(probability=0.3, grid_width=4, grid_height=4, magnitude=6)
    p.random_brightness(probability=0.8, min_factor=0.5, max_factor=2)
    p.random_color(probability=0.3, min_factor=0.5, max_factor=1.5)
    p.random_contrast(probability=0.3, min_factor=0.8, max_factor=1.2)

    p.sample(n_samples, multi_threaded=False)

In [ ]:
# Remove previous augmented images, if any (in case you run it twice) 

import shutil

f_augmented = os.path.join(f_dir_sel, 'output')
m_augmented = os.path.join(m_dir_sel, 'output')

if os.path.exists(f_augmented):
    shutil.rmtree(f_augmented)
    
if os.path.exists(m_augmented):
    shutil.rmtree(m_augmented)

In [ ]:
print(f_augmented, m_augmented)

In [ ]:
# Number of augmented images

n_f = 1000 # number of augmented female images
n_m = 1000 # number of augmented male images

# Task: choose your own number of augmented images for the two classes

# Hint 1: if you start with smaller numbers, the CNN will be trained faster
# However, the more images you add, the better accuracy/performace you will achieve
# The performance of the CNN can be evaluated in the section "Prediction Performance"
# Hint 2: consider whether you want to select the same or an unequal number of 
# images for the two classes

# Create augmented images
build_augmented_images(f_dir_sel, n_f)
build_augmented_images(m_dir_sel, n_m)  

In [ ]:
# Check number of augmented images of each category

print_file_count_in_dir(f_augmented, "female augmented cases: ")
print_file_count_in_dir(m_augmented, "male augmented cases: ")

In [ ]:
# Show example augmented images
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io
import os

import fnmatch

def show_augmented_images_example(image_type, person):
  
  base_path = os.path.join(os.path.join('faces', image_type.lower()),'selected')
  augmentation_path = os.path.join(base_path, 'output')

  pattern = '*' + person + '*.jpg';
  original_match = fnmatch.filter(os.listdir(base_path), pattern)
  if len(original_match) <= 0:
    print("select a valid image number or type for: " + person)
    return

  original = io.imread(os.path.join(base_path, original_match[0]))
  augmented = fnmatch.filter(os.listdir(augmentation_path), pattern)

  n_augmented = len(augmented)

  fig = plt.figure(figsize=(20,10))
  ax1 = fig.add_subplot(1,n_augmented+1,1)
  ax1.set_title("Original")
  ax1.imshow(original)

  for a in range(n_augmented):
    im_augmented = io.imread(os.path.join(augmentation_path, augmented[a]))
    ax = fig.add_subplot(1,n_augmented+1,a+2)
    ax.imshow(im_augmented)
    ax.set_title("Augmented")


In [ ]:
!ls $f_augmented

In [ ]:
# Task: display exemplar augmented images: change image number and image type
person = 'Conchita_Martinez_0001' # e.g., Vanessa_Redgrave_0002, Yoriko_Kawaguchi_0004
image_type = 'female' # 'female' or 'male'
show_augmented_images_example(image_type, person)

Then continue with the section data preparation.